# Relax Challenge

###### The  data  is  available  as  two  attached  CSV  files:
* takehome_user_engagement. csv
* takehome_users . csv
###### The  data  has  the  following  two  tables:

1]  A  user  table  ( "takehome_users" )  with  data  on  12,000  users  who  signed  up  for  the
product  in  the  last  two  years.   
###### This  table  includes:
● name:  the  user's  name

● object_id:   the  user's  id

● email:  email  address

● creation_source:   how  their  account  was  created.  This  takes  on  one
of  5  values:

    * PERSONAL_PROJECTS:  invited  to  join  another  user's personal  workspace
    * GUEST_INVITE:  invited  to  an  organization  as  a  guest (limited  permissions)
    * ORG_INVITE:  invited  to  an  organization  (as  a  full  member)
    * SIGNUP:  signed  up  via  the  website
    * SIGNUP_GOOGLE_AUTH:  signed  up  using  Google Authentication  (using  a  Google  email  account  for  their  login
id)

● creation_time:  when  they  created  their  account

● last_session_creation_time:   unix  timestamp  of  last  login

● opted_in_to_mailing_list:  whether  they  have  opted  into  receiving
marketing  emails

● enabled_for_marketing_drip:  whether  they  are  on  the  regular
marketing  email  drip

● org_id:   the  organization  (group  of  users)  they  belong  to

● invited_by_user_id:   which  user  invited  them  to  join  (if  applicable). 

2]  A  usage  summary  table  ( "takehome_user_engagement" )  that  has  a  row  for  each  day
that  a  user  logged  into  the  product.
Defining  an  "adopted  user"   as  a  user  who   has  logged  into  the  product  on  three  separate
days  in  at  least  one  seven­day  period ,  identify  which  factors  predict  future  user
adoption .

We  suggest  spending  1­2  hours  on  this,  but  you're  welcome  to  spend  more  or  less.
Please  send  us  a  brief  writeup  of  your  findings  (the  more  concise,  the  better  ­­  no  more
than  one  page),  along  with  any  summary  tables,  graphs,  code,  or  queries  that  can  help
us  understand  your  approach.  Please  note  any  factors  you  considered  or  investigation
you  did,  even  if  they  did  not  pan  out.  Feel  free  to  identify  any  further  research  or  data
you  think  would  be  valuable.

In [1]:
import pandas as pd
import gzip
import matplotlib.pyplot as plt
import datetime
import calendar
import numpy as np
from string import punctuation
from __future__ import division
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize
from nltk.tokenize import regexp_tokenize
import re
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
plt.style.use('fivethirtyeight')
from IPython.core.debugger import Pdb
import time
from datetime import datetime
from dateutil.parser import parse

#### Part 1 - EDA

In [2]:
df_users = pd.read_csv('takehome_users.csv', encoding='latin-1')

In [3]:
df_user_engagement = pd.read_csv('takehome_user_engagement.csv')

In [5]:
pd.unique(df_users.creation_source.values)

array(['GUEST_INVITE', 'ORG_INVITE', 'SIGNUP', 'PERSONAL_PROJECTS',
       'SIGNUP_GOOGLE_AUTH'], dtype=object)

In [6]:
df_users.groupby('creation_source').count() 

,object_id,creation_time,name,email,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
creation_source,,,,,,,,,
GUEST_INVITE,2163,2163,2163,2163,1588,2163,2163,2163,2163
ORG_INVITE,4254,4254,4254,4254,3188,4254,4254,4254,4254
PERSONAL_PROJECTS,2111,2111,2111,2111,764,2111,2111,2111,0
SIGNUP,2087,2087,2087,2087,1898,2087,2087,2087,0
SIGNUP_GOOGLE_AUTH,1385,1385,1385,1385,1385,1385,1385,1385,0


In [72]:
# df_users.groupby('invited_by_user_id').count()

In [8]:
df_users.describe()

C:\Program Files\Anaconda33\lib\site-packages\numpy\lib\function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
count,12000.00000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000
mean,6000.50000,1.379279e+09,0.249500,0.149333,141.884583,5962.957145
std,3464.24595,1.953116e+07,0.432742,0.356432,124.056723,3383.761968
min,1.00000,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,NaN,0.000000,0.000000,29.000000,NaN
50%,6000.50000,NaN,0.000000,0.000000,108.000000,NaN
75%,9000.25000,NaN,0.000000,0.000000,238.250000,NaN
max,12000.00000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000


In [9]:
df_user_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [10]:
# How many users whose last session creation time is null = 3177/12000
df_users.last_session_creation_time.isnull().values.ravel().sum()

3177

In [11]:
# How many users were not invited by another user = 5583/12000
df_users.invited_by_user_id.isnull().values.ravel().sum()

5583

In [12]:
df_users.last_session_creation_time.fillna(method='ffill',inplace=True)

In [13]:
df_users.last_session_creation_time = df_users.last_session_creation_time.astype(int)

In [14]:
# !pip install simple-date

In [15]:
df_users.invited_by_user_id.fillna(method='ffill',inplace=True)

In [16]:
df_users.describe()

,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
count,12000.00000,1.200000e+04,12000.000000,12000.000000,12000.000000,12000.000000
mean,6000.50000,1.379384e+09,0.249500,0.149333,141.884583,5928.445167
std,3464.24595,1.953509e+07,0.432742,0.356432,124.056723,3396.672604
min,1.00000,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,1.363310e+09,0.000000,0.000000,29.000000,2994.000000
50%,6000.50000,1.383154e+09,0.000000,0.000000,108.000000,5909.500000
75%,9000.25000,1.398533e+09,0.000000,0.000000,238.250000,8841.000000
max,12000.00000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000


In [17]:
df_users.head(100)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1398138810,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1396237504,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1363734892,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1369210168,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1358849660,0,0,193,5240.0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1387424226,0,0,197,11241.0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1356009872,0,1,37,11241.0
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,1356009872,1,1,74,11241.0
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,1356009872,0,0,302,11241.0
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1401833283,1,1,318,4143.0


In [18]:
import datetime
new_dt = (
    datetime.datetime.fromtimestamp(
        int("1396238000")
    ).strftime('%Y-%m-%d %H:%M:%S'))
print(new_dt)

2014-03-30 23:53:20


In [19]:
if pd.to_datetime(new_dt) > pd.to_datetime(df_users.creation_time[1]):
    delta_dt = pd.to_datetime(new_dt) - pd.to_datetime(df_users.creation_time[1])
    print("last session login after account creation")
else:
    delta_dt = pd.to_datetime(df_users.creation_time[1]) - pd.to_datetime(new_dt)
    print("account created after last session login")
print(delta_dt)

last session login after account creation
135 days 20:08:16


In [20]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    12000 non-null int32
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            12000 non-null float64
dtypes: float64(1), int32(1), int64(4), object(4)
memory usage: 890.7+ KB


In [21]:
df_user_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [22]:
df_new_eng = df_user_engagement.groupby(["user_id", "visited"]).size().reset_index(name='count')
df_eng_mult_visits = df_new_eng[df_new_eng["count"] > 2]
df_eng_mult_visits.head()

,user_id,visited,count
1,2,1,14
7,10,1,284
13,20,1,7
24,33,1,18
28,42,1,342


In [23]:
df_eng_mult_visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2248 entries, 1 to 8815
Data columns (total 3 columns):
user_id    2248 non-null int64
visited    2248 non-null int64
count      2248 non-null int64
dtypes: int64(3)
memory usage: 70.2 KB


In [73]:
new_eng = pd.merge(df_user_engagement,df_eng_mult_visits, on="user_id")
# new_eng

In [25]:
new_eng.drop(['visited_y','count'],axis=1,inplace=True)

In [30]:
# for uid in new_eng.user_id:
new_eng_copy = new_eng.copy()
final_df = pd.DataFrame()
final_dict = []

for uid in df_eng_mult_visits.user_id:
    new_eng_by_user = new_eng.loc[new_eng.user_id == uid]
#     print(new_eng_by_user)
    new_eng_by_user.time_stamp = pd.to_datetime(new_eng_by_user.time_stamp)
#     print(new_eng_by_user.info())
    new_eng_by_user.set_index('time_stamp', inplace=True)
#     print(new_eng_by_user.info())
#     print(new_df)
    new_df = new_eng_by_user.visited_x.resample('7D').count().reset_index(name='total_visits')
#     print(new_df.info())
    new_df['user_id'] = uid
    new_df['adopted_user'] = [1 if tvisit >= 3 else 0 for tvisit in new_df.total_visits]
    if 1 in pd.unique(new_df.adopted_user.values):
        final_dict.append(uid)
print(final_dict)
    
#     print(new_df)

C:\Program Files\Anaconda33\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


[2, 10, 20, 42, 43, 53, 63, 69, 74, 80, 81, 82, 87, 133, 135, 141, 146, 153, 160, 165, 168, 172, 174, 185, 188, 197, 200, 202, 203, 209, 214, 230, 245, 247, 263, 265, 275, 280, 283, 297, 298, 305, 310, 311, 322, 341, 347, 351, 363, 370, 383, 401, 418, 430, 445, 450, 460, 462, 469, 471, 472, 479, 483, 492, 494, 497, 509, 510, 512, 518, 522, 529, 535, 540, 547, 553, 564, 572, 589, 591, 601, 603, 605, 618, 627, 628, 632, 634, 639, 669, 679, 680, 724, 725, 728, 754, 772, 783, 786, 804, 845, 851, 869, 874, 882, 885, 901, 906, 907, 912, 928, 932, 934, 937, 943, 953, 980, 985, 1007, 1009, 1013, 1017, 1018, 1026, 1027, 1055, 1061, 1072, 1089, 1093, 1094, 1099, 1106, 1107, 1119, 1123, 1124, 1128, 1129, 1136, 1145, 1150, 1151, 1155, 1156, 1163, 1186, 1196, 1202, 1214, 1233, 1235, 1242, 1245, 1250, 1274, 1280, 1290, 1303, 1318, 1319, 1320, 1327, 1339, 1343, 1345, 1350, 1357, 1361, 1368, 1379, 1396, 1407, 1410, 1411, 1421, 1434, 1464, 1472, 1476, 1479, 1480, 1485, 1506, 1507, 1508, 1525, 1546, 155

In [34]:
len(final_dict)

1472

In [35]:
df_users

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1398138810,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1396237504,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1363734892,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1369210168,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1358849660,0,0,193,5240.0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1387424226,0,0,197,11241.0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1356009872,0,1,37,11241.0
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,1356009872,1,1,74,11241.0
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,1356009872,0,0,302,11241.0
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1401833283,1,1,318,4143.0


In [47]:
model_data = df_users.copy()

In [48]:
model_data = model_data.drop(['creation_time','name','email','last_session_creation_time','invited_by_user_id','org_id'], axis=1)

In [69]:
model_data['adopted_user'] = [1 if x in final_dict else 0 for x in model_data.object_id]

print(model_data)

       object_id     creation_source  opted_in_to_mailing_list  \
0              1        GUEST_INVITE                         1   
1              2          ORG_INVITE                         0   
2              3          ORG_INVITE                         0   
3              4        GUEST_INVITE                         0   
4              5        GUEST_INVITE                         0   
5              6        GUEST_INVITE                         0   
6              7              SIGNUP                         0   
7              8   PERSONAL_PROJECTS                         1   
8              9   PERSONAL_PROJECTS                         0   
9             10          ORG_INVITE                         1   
10            11              SIGNUP                         0   
11            12          ORG_INVITE                         0   
12            13          ORG_INVITE                         0   
13            14              SIGNUP                         0   
14        

In [70]:
print(model_data.groupby('adopted_user').count())

              object_id  creation_source  opted_in_to_mailing_list  \
adopted_user                                                         
0                 10528            10528                     10528   
1                  1472             1472                      1472   

              enabled_for_marketing_drip  
adopted_user                              
0                                  10528  
1                                   1472  


###### Take the above dataset and applying the one-hot encoding for the 'creation-source' to make it a numerical field would be my next steps.  Followed by conducting the train-test split and work on the data as a binary identification of the 'adopted user' . Use of the logistic regression model to create a model and finally conducting a predictive analysis on the test set to give us 'adopted user' values would be necessary.

###### A time-series analysis could also be conducted of the account creation time versus the last session creation time. This should tell us about how engaged the user is and how recently a session was creation showcasing their loyalty.

###### A final visual analysis of the actual versus predicted values provides a good accuracy measure.  A need-based next step would be to conduct a boosting analysis where one could boost the accuracy numbers of the model by using AdaBoosting, XGBoost or GradientBoosting algorithms.